In [1]:
using Makie
using Random

In [2]:
function forceNBody3D(x,y,z,N,k;dl=1., m=1., g=9.8)
    dx = copy(x)
    dy = copy(y)
    dz = copy(z)
    
    for i in N:-1:2
        dx[i] -= dx[i-1]
        dy[i] -= dy[i-1]
        dz[i] -= dz[i-1]
    end
    
    l = sqrt.(dx.^2 + dy.^2 + dz.^2)
    #println("長さ",l)
    θ = acos.(dz./l)
    ϕ = atan.(dy, dx)
    #lxy = sqrt.(dx.^2 + dy.^2)
    
    ax = Vector{Float64}(undef,N)
    ay = Vector{Float64}(undef,N)
    az = Vector{Float64}(undef,N)
    
    for i in 1:N-1
        ax[i] = (
            -k*(l[i]-dl)*sin(θ[i])*cos(ϕ[i]) 
            +k*(l[i+1]-dl)*sin(θ[i+1])*cos(ϕ[i+1])
            )
        
        ay[i] = (
            -k*(l[i]-dl)*sin(θ[i])*sin(ϕ[i]) 
            +k*(l[i+1]-dl)*sin(θ[i+1])*sin(ϕ[i+1]) 
            )
        az[i] = (
            -k*(l[i]-dl)*cos(θ[i])
            +k*(l[i+1]-dl)*cos(θ[i+1])
            -g
            )
    end   
    ax[N] = -k*(l[N]-dl)*sin(θ[N])*cos(ϕ[N])
    ay[N] = -k*(l[N]-dl)*sin(θ[N])*sin(ϕ[N]) 
    az[N] = -k*(l[N]-dl)*cos(θ[N]) - g   
    
    #println("ax",ax)
    #println("ay",ay)
    #println("az",az)
    return ax, ay, az
end

function forceNBody3D(x,y,z,u,v,w,N,k,γ;dl=1., m=1., g=9.8)
    dx = copy(x)
    dy = copy(y)
    dz = copy(z)
    
    for i in N:-1:2
        dx[i] -= dx[i-1]
        dy[i] -= dy[i-1]
        dz[i] -= dz[i-1]
    end
    
    l = sqrt.(dx.^2 + dy.^2 + dz.^2)
    θ = acos.(dz./l)
    ϕ = atan.(dy, dx)
    #lxy = sqrt.(dx.^2 + dy.^2)
    
    ax = Vector{Float64}(undef,N)
    ay = Vector{Float64}(undef,N)
    az = Vector{Float64}(undef,N)
    
    for i in 1:N-1
        ax[i] = (
            -k*(l[i]-dl)*sin(θ[i])*cos(ϕ[i]) 
            +k*(l[i+1]-dl)*sin(θ[i+1])*cos(ϕ[i+1])
            -γ*u[i]
            )
        
        ay[i] = (
            -k*(l[i]-dl)*sin(θ[i])*sin(ϕ[i]) 
            +k*(l[i+1]-dl)*sin(θ[i+1])*sin(ϕ[i+1]) 
            -γ*v[i]
        )
        az[i] = (
            -k*(l[i]-dl)*cos(θ[i])
            +k*(l[i+1]-dl)*cos(θ[i+1])
            -g
            -γ*w[i]
            )
    end   
    ax[N] = -k*(l[N]-dl)*sin(θ[N])*cos(ϕ[N]) -γ*u[N]
    ay[N] = -k*(l[N]-dl)*sin(θ[N])*sin(ϕ[N]) -γ*v[N]
    az[N] = -k*(l[N]-dl)*cos(θ[N]) - g -γ*w[N]       
    return ax, ay, az
end




forceNBody3D (generic function with 2 methods)

# ルンゲクッタ N体 3D

In [3]:
function ルンゲクッタ(θ,ϕ,ω,Ω,N; T=20., dt=1e-3,k=1e3, dl=1.)
    m = 1.
    g = 9.8
    
    numStep = length(0:dt:T)
    
    x = dl.*sin.(θ).*cos.(ϕ)
    y = dl.*sin.(θ).*sin.(ϕ)
    z = dl.*cos.(θ)
    
    u = dl.*ω.*cos.(θ).*cos.(ϕ) - dl.*Ω.*sin.(θ).*sin.(ϕ) 
    v = dl.*ω.*cos.(θ).*sin.(ϕ) + dl.*Ω.*sin.(θ).*cos.(ϕ)
    w = -dl.*ω.*sin.(θ)
    
    for i in 2:N
        x[i]+=x[i-1]
        y[i]+=y[i-1]
        z[i]+=z[i-1]
        u[i]+=u[i-1]
        v[i]+=v[i-1]
        w[i]+=w[i-1]
    end
    
    xlist = Array{Float64,2}(undef,N,numStep)
    ylist = Array{Float64,2}(undef,N,numStep)
    zlist = Array{Float64,2}(undef,N,numStep)
    ulist = Array{Float64,2}(undef,N,numStep)
    vlist = Array{Float64,2}(undef,N,numStep)
    wlist = Array{Float64,2}(undef,N,numStep)
    
    
    for (i,t) in enumerate(0:dt:T)
        # リストに書き込む
        xlist[:,i] = x
        ylist[:,i] = y
        zlist[:,i] = z
        ulist[:,i] = u
        vlist[:,i] = v
        wlist[:,i] = w
        
        # 微小変化の計算
        # k1
        dx1 = u
        dy1 = v
        dz1 = w
        du1, dv1, dw1 = forceNBody3D(x,y,z,N,k)
        
        # k2
        x1 = x .+ .5dt.*dx1 
        y1 = y .+ .5dt.*dy1
        z1 = z .+ .5dt.*dz1
        u1 = u .+ .5dt.*du1
        v1 = v .+ .5dt.*dv1
        w1 = w .+ .5dt.*dw1
        
        dx2 = u1
        dy2 = v1
        dz2 = w1
        du2, dv2, dw2 = forceNBody3D(x1,y1,z1,N,k)
        
        # k3
        x2 = x .+ .5dt.*dx2 
        y2 = y .+ .5dt.*dy2
        z2 = z .+ .5dt.*dz2
        u2 = u .+ .5dt.*du2
        v2 = v .+ .5dt.*dv2
        w2 = w .+ .5dt.*dw2
        
        dx3 = u2
        dy3 = v2
        dz3 = w2
        du3, dv3, dw3 = forceNBody3D(x2,y2,z2,N,k)
        
        # k4
        x3 = x + dt*dx3 
        y3 = y + dt*dy3
        z3 = z + dt*dz3
        u3 = u + dt*du3
        v3 = v + dt*dv3
        w3 = w + dt*dw3
        
        dx4 = u3
        dy4 = v3
        dz4 = w3
        du4, dv4, dw4 = forceNBody3D(x3,y3,z3,N,k)
        
        # がっちゃんこ
        x += dt*(dx1 + 2dx2 + 2dx3 + dx4)/6. 
        y += dt*(dy1 + 2dy2 + 2dy3 + dy4)/6.
        z += dt*(dz1 + 2dz2 + 2dz3 + dz4)/6.
        u += dt*(du1 + 2du2 + 2du3 + du4)/6.
        v += dt*(dv1 + 2dv2 + 2dv3 + dv4)/6.
        w += dt*(dw1 + 2dw2 + 2dw3 + dw4)/6.
        
    end
    return 0:dt:T, xlist, ylist,zlist, ulist, vlist,wlist
end

function ルンゲクッタ(θ,ϕ,ω,Ω,N,γ ; T=20., dt=1e-3,k=1e3,dl)
   
    m = 1.
    g = 9.8
    
    numStep = length(0:dt:T)
    
    x = dl*sin.(θ).*cos.(ϕ)
    y = dl*sin.(θ).*sin.(ϕ)
    z = dl*cos.(θ)
    
    u = dl*ω.*cos.(θ).*cos.(ϕ) - dl*Ω.*sin.(θ).*sin.(ϕ) 
    v = dl*ω.*cos.(θ).*sin.(ϕ) + dl*Ω.*sin.(θ).*cos.(ϕ)
    w = -dl*ω.*sin.(θ)
    
    for i in 2:N
        x[i]+=x[i-1]
        y[i]+=y[i-1]
        z[i]+=z[i-1]
        u[i]+=u[i-1]
        v[i]+=v[i-1]
        w[i]+=w[i-1]
    end
    
    xlist = Array{Float64,2}(undef,N,numStep)
    ylist = Array{Float64,2}(undef,N,numStep)
    zlist = Array{Float64,2}(undef,N,numStep)
    ulist = Array{Float64,2}(undef,N,numStep)
    vlist = Array{Float64,2}(undef,N,numStep)
    wlist = Array{Float64,2}(undef,N,numStep)
    
    
    for (i,t) in enumerate(0:dt:T)
        # リストに書き込む
        xlist[:,i] = x
        ylist[:,i] = y
        zlist[:,i] = z
        ulist[:,i] = u
        vlist[:,i] = v
        wlist[:,i] = w
        
        # 微小変化の計算
        # k1
        dx1 = u
        dy1 = v
        dz1 = w
        du1, dv1, dw1 = forceNBody3D(x,y,z,u,v,w,N,k,γ)
        
        # k2
        x1 = x + .5dt*dx1 
        y1 = y + .5dt*dy1
        z1 = z + .5dt*dz1
        u1 = u + .5dt*du1
        v1 = v + .5dt*dv1
        w1 = w + .5dt*dw1
        
        dx2 = u1
        dy2 = v1
        dz2 = w1
        du2, dv2, dw2 = forceNBody3D(x1,y1,z1,u1,v1,w1,N,k,γ)
        
        # k3
        x2 = x + .5dt*dx2 
        y2 = y + .5dt*dy2
        z2 = z + .5dt*dz2
        u2 = u + .5dt*du2
        v2 = v + .5dt*dv2
        w2 = w + .5dt*dw2
        
        dx3 = u2
        dy3 = v2
        dz3 = w2
        du3, dv3, dw3 = forceNBody3D(x2,y2,z2,u2,v2,w2,N,k,γ)
        
        # k4
        x3 = x + dt*dx3 
        y3 = y + dt*dy3
        z3 = z + dt*dz3
        u3 = u + dt*du3
        v3 = v + dt*dv3
        w3 = w + dt*dw3
        
        dx4 = u3
        dy4 = v3
        dz4 = w3
        du4, dv4, dw4 = forceNBody3D(x3,y3,z3,u3,v3,w3,N,k,γ)
        
        # がっちゃんこ
        x += dt*(dx1 + 2dx2 + 2dx3 + dx4)/6. 
        y += dt*(dy1 + 2dy2 + 2dy3 + dy4)/6.
        z += dt*(dz1 + 2dz2 + 2dz3 + dz4)/6.
        u += dt*(du1 + 2du2 + 2du3 + du4)/6.
        v += dt*(dv1 + 2dv2 + 2dv3 + dv4)/6.
        w += dt*(dw1 + 2dw2 + 2dw3 + dw4)/6.
        
    end
    return 0:dt:T, xlist, ylist,zlist, ulist, vlist,wlist
end



ルンゲクッタ (generic function with 2 methods)

In [24]:

function movie(t,x,y,z,N;step=1,scale=1.,marker=1.)
    dl= 1.
    
    scene = Scene(resolution = (1200,1200))
    #scene = linesegments!(scene, FRect3D(Vec3f0(-scale*N,-scale*N,-scale*N), Vec3f0(2*scale*N,2*scale*N,2*scale*N)))
    #update_cam!(scene,Vec3f0(0.001,0.01,.001),Vec3f0(0.,0,1),Vec3f0(0.,0,1))
    #update_cam!(scene, Vec3f0())
    mystep = Node(1)

    scene = meshscatter!(
        scene, 
        lift(i -> x[:,i], mystep),
        lift(i -> y[:,i], mystep),
        lift(i -> z[:,i], mystep),

        markersize = marker*dl,
        #color = :blue,
        color = to_colormap(:viridis,N),
        #
        limits =FRect3D(Vec3f0(-scale*N,-scale*N,-scale*N), Vec3f0(2*scale*N,2*scale*N,2*scale*N))
    )
        

    eyepos = Vec3f0(0,3N,.25N)
    record(scene, "yossha7.mp4",range(1,length(t),step=step);framerate=60) do i
        #println(i)
        mystep[]=i
        eyepos = Vec3f0(2N*sin(i/1e4), -2N*cos(i/1e4),eyepos[3])
        update_cam!(scene, eyepos, Vec3f0(0,0, -0.4N))
        
        end;
    
end

movie (generic function with 1 method)

In [25]:
N = 500
θ = π * rand(N)
ϕ = 2π * rand(N)
ω = 8π * (rand(N) .- .5)
Ω = 8π * (rand(N) .- .5)


@time t,x,y,z,u,v,w = ルンゲクッタ(θ, ϕ,ω, Ω, N,;T=50, dt=1e-3, k=1e5);
@time movie(t,x,y,z, N; marker=2.5,scale=1, step=10)

 19.751945 seconds (7.20 M allocations: 22.866 GiB, 12.21% gc time)
232.341861 seconds (3.91 M allocations: 20.390 GiB, 1.37% gc time)


"yossha7.mp4"